In [1]:
import numpy as np
import cv2 
import sys 
import time
import os

In [2]:
import os
 
def mkdir(path):
 
    folder = os.path.exists(path)
 
    if not folder:                   #判断是否存在文件夹如果不存在则创建为文件夹
        os.makedirs(path)            #makedirs 创建文件时如果路径不存在会创建这个路径
        print("---  new folder...  ---")
        print("---  OK  ---")
         
    else:
        print("---  There is this folder!  ---" + path)


In [3]:
# Create sub folders from source folder
source_folder = "graph_file"
for root, dirs, files in os.walk(source_folder, topdown=False):
#     for name in files:
#         if '.graph' in name:
#             print(os.path.join(root, name))
    for name in dirs:
        file_path = os.path.join(root, name)
        if "graph_file/Uniform" == file_path or "graph_file/Sparse" == file_path:
            continue
        print(file_path.replace("graph","jpg"))
        mkdir(file_path.replace("graph","jpg"))

jpg_file/Uniform/FRU
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/FRR
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/DH
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/KK
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/FA2
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/LINLOG
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/FDGE
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/KK_W
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/FR
---  new folder...  ---
---  OK  ---
jpg_file/Uniform/JIGGLE
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/FRU
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/FRR
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/DH
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/KK
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/FA2
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/LINLOG
---  new folder...  ---
---  OK  ---
jpg_file/Sparse/FDGE
---  new folder...  ---
---  OK  ---
jpg_file/Spars

In [4]:
# color
color_of_edges = (0, 0, 0) # black
color_of_holes = (255, 255, 255) # white
color_of_nodes = (0, 0, 0)

#Read data file
def readData(fname):
    data_list = [] #The data list
    num_nodes = int
    all_nodes = []
    num_edges = int
    all_edges = []
    node_x = []
    node_y = []
    data_f = open(fname, 'r')
    for line in data_f.readlines():
        data_list.append(line.strip())
    
    num_nodes = int (data_list.pop(0)) #Pop out the number of nodes 
    #Get all the number node coordinate
    for i in range (0,num_nodes):
        node = data_list.pop(0) #Pop out the node coordinate
        node = node.split()
        node.pop(0) #Pop out the node index  
        node_x.append(node[0])
        node_y.append(node[1])
    #Change it to list    
    node_x = list(map(float, node_x))
    node_y = list(map(float, node_y))
    #Find the minimun value in x and y list
    min_x = min(node_x)
    min_y = min(node_y)
    #If the coorodinate of nodes are nagative, change it to be positive
    if min_x or min_y < 0:
        for i in range (0,num_nodes):
            node_x[i] = node_x[i] + min_x * -1
            node_y[i] = node_y[i] + min_y * -1
    #Find the maximun value in x and y list        
    max_x = max(node_x)
    max_y = max(node_y)
    #Calulate the canvas size by the node numbers and radius of node
    canvas_size = (4 * num_nodes)
    #Calulate the ratio for enlarge the coorodinate of node that fitting the canvas size
    node_ratio = (min((canvas_size / max_x), (canvas_size / max_y)))
    #print(node_ratio)
    #Origin coordinates
    xc = 0
    yc = 0
    
    for i in range (0,num_nodes):
        x = int(xc + node_ratio * (node_x[i] - xc))
        y = int(yc + node_ratio * (node_y[i] - yc))
        all_nodes.append((x, y)) #(X, Y) 
    
    num_edges = int (data_list.pop(0)) #Pop out the number of edges 
    #Get all the edge
    for i in range (0,num_edges): 
        edge = data_list.pop(0) #Pop out the edge connection
        edge = edge.split()
        all_edges.append([int(edge[0]),int(edge[1])]) #(point 1, point 2)
    data_f.close
    
    area_min_radio = 5000 / 4000
    area_min = area_min_radio * canvas_size
    
    return all_nodes, all_edges, num_nodes, canvas_size, area_min

In [5]:
#Lable the hole
def holeLabel(img,hole,t = "Hole"):
    M = cv2.moments(hole)
    cX = int(M['m10'] / M['m00'])
    cY = int(M['m01'] / M['m00'])
    cv2.putText(img,str(t), (cX,cY),cv2.FONT_HERSHEY_SIMPLEX, 1,(240,240,240),3,cv2.LINE_AA)
    return img

#Draw all the nodes
def drawAllNodes(img,all_nodes,radiu,color = color_of_nodes,thickness=1,lineType=0,shift=0):
    for i in range (0,int(len(all_nodes))): 
        cv2.circle(img, all_nodes[i], radiu ,color,thickness,lineType,shift)  #Add the node result
        
    return img

#Draw all the edges
def drawAlledges(img,all_nodes,all_edges,color = color_of_edges,thickness=3,lineType=8,shift=0):
    for i in range (0,int(len(all_edges))):
        cv2.line(img, all_nodes[all_edges[i][0]], all_nodes[all_edges[i][1]],color,thickness, lineType, shift)
        
    return img

#Find the nodes of hole
def findHoleNodes(hole,all_nodes,img = " ",color = color_of_nodes,thickness = 3,label = False):
    nodes_lst = []  #The nodes of hole 
    a = 0.0         #The return value of distance which between the nodes and hole in the image  
    d = -4.99       #The distance which between the nodes and hole
    for i in range (len(all_nodes)):
        a = cv2.pointPolygonTest(hole,all_nodes[i],True) 
        if (a >= d):
            nodes_lst.append(i)
            if (label == True):
                    cv2.circle(img, all_nodes[i], 14,color,thickness)  #Add the node result
                                      
    return nodes_lst

#Find all the nodes of holes
def findAllHolesNodes(all_holes,all_nodes,img = " ",color = color_of_nodes,thickness = 3,label = False,sf = False):
    holes_nodes = []  #The nodes of hole
    with open(output_file_name ,'w') as f:
        f.write("%s\n"%num_nodes)
        for i in range (0,int(len(all_holes))):
            holes_nodes.append(findHoleNodes(all_holes[i],all_nodes,img,color,thickness,label))
            f.write("%s\n"%(holes_nodes[i][:]))
    return holes_nodes,img

#Main Program:
def generateJPG(fname):
    
    time_start = time.time()
    all_nodes = []
    all_edges = []
    
    #Read data
    all_nodes, all_edges, num_nodes, canvas_size, area_min = readData(fname) 
    
    # print(all_nodes)
    results_img = np.zeros((canvas_size,canvas_size, 3), np.uint8) #create a gray img
    img = np.full((canvas_size, canvas_size, 3), 255 ,np.uint8) #create a img
    
    #Add all the edges    
    drawAlledges(results_img,all_nodes,all_edges,color_of_edges)

    results_img = img.copy()

    #Draw all the edge
    drawAlledges(results_img,all_nodes,all_edges,color_of_edges,lineType= cv2.LINE_AA)
    drawAlledges(img,all_nodes,all_edges,color_of_edges,lineType= cv2.LINE_AA)

    #Draw all the node
    drawAllNodes(img,all_nodes,1,color_of_nodes,lineType= cv2.LINE_AA);
    drawAllNodes(results_img,all_nodes,1,color_of_nodes,lineType= cv2.LINE_AA);

    cv2.imwrite(fname.replace("graph","jpg"), results_img)
    time_end = time.time()
    print('It cost %f seconds' % (time_end - time_start))
    print ("DONE.")


In [6]:
for root, dirs, files in os.walk("./graph_file", topdown=False):
    for name in files:
        if '.graph' in name:
            generateJPG(os.path.join(root, name))
            print(os.path.join(root, name))
#     for name in dirs:
#         print(os.path.join(root, name))

It cost 0.662572 seconds
DONE.
./graph_file/Uniform/FRU/n=2000d=6_1892.graph
It cost 0.047744 seconds
DONE.
./graph_file/Uniform/FRU/n=500d=6_25872.graph
It cost 0.068527 seconds
DONE.
./graph_file/Uniform/FRU/n=500d=10_24825.graph
It cost 0.759977 seconds
DONE.
./graph_file/Uniform/FRU/n=2000d=8_1881.graph
It cost 0.288570 seconds
DONE.
./graph_file/Uniform/FRU/n=1000d=12_7881.graph
It cost 1.926392 seconds
DONE.
./graph_file/Uniform/FRU/n=3000d=10_853.graph
It cost 2.406455 seconds
DONE.
./graph_file/Uniform/FRU/n=3000d=15_858.graph
It cost 0.109594 seconds
DONE.
./graph_file/Uniform/FRU/n=500d=15_19464.graph
It cost 1.382756 seconds
DONE.
./graph_file/Uniform/FRU/n=3000d=6_932.graph
It cost 1.943580 seconds
DONE.
./graph_file/Uniform/FRU/n=3000d=12_859.graph
It cost 1.265227 seconds
DONE.
./graph_file/Uniform/FRU/n=2000d=15_1746.graph
It cost 0.879344 seconds
DONE.
./graph_file/Uniform/FRU/n=2000d=12_1914.graph
It cost 0.859385 seconds
DONE.
./graph_file/Uniform/FRU/n=2000d=10_1739.

It cost 21.237595 seconds
DONE.
./graph_file/Uniform/LINLOG/n=3000d=6_2186.graph
It cost 15.775346 seconds
DONE.
./graph_file/Uniform/LINLOG/n=2000d=10_3645.graph
It cost 12.773011 seconds
DONE.
./graph_file/Uniform/LINLOG/n=2000d=12_2985.graph
It cost 2.936632 seconds
DONE.
./graph_file/Uniform/LINLOG/n=1000d=15_6578.graph
It cost 33.610786 seconds
DONE.
./graph_file/Uniform/LINLOG/n=3000d=10_2105.graph
It cost 2.693635 seconds
DONE.
./graph_file/Uniform/LINLOG/n=1000d=8_8738.graph
It cost 2.948603 seconds
DONE.
./graph_file/Uniform/LINLOG/n=1000d=12_7526.graph
It cost 2.501116 seconds
DONE.
./graph_file/Uniform/LINLOG/n=1000d=10_7836.graph
It cost 0.439271 seconds
DONE.
./graph_file/Uniform/LINLOG/n=500d=8_19938.graph
It cost 1.396495 seconds
DONE.
./graph_file/Uniform/LINLOG/n=1000d=6_8982.graph
It cost 31.978770 seconds
DONE.
./graph_file/Uniform/LINLOG/n=3000d=8_2362.graph
It cost 21.936922 seconds
DONE.
./graph_file/Uniform/LINLOG/n=2000d=15_2947.graph
It cost 53.550695 seconds
D

It cost 0.219033 seconds
DONE.
./graph_file/Sparse/FRU/n=1000d=12_8079.graph
It cost 0.750429 seconds
DONE.
./graph_file/Sparse/FRU/n=2000d=12_1992.graph
It cost 0.158432 seconds
DONE.
./graph_file/Sparse/FRU/n=1000d=8_7551.graph
It cost 0.191965 seconds
DONE.
./graph_file/Sparse/FRU/n=1000d=10_7855.graph
It cost 0.484956 seconds
DONE.
./graph_file/Sparse/FRU/n=2000d=6_1958.graph
It cost 0.597598 seconds
DONE.
./graph_file/Sparse/FRU/n=2000d=8_2007.graph
It cost 0.119179 seconds
DONE.
./graph_file/Sparse/FRU/n=1000d=6_7910.graph
It cost 13.162205 seconds
DONE.
./graph_file/Sparse/FRR/n=2000d=15_1807.graph
It cost 10.380104 seconds
DONE.
./graph_file/Sparse/FRR/n=2000d=12_1716.graph
It cost 2.283955 seconds
DONE.
./graph_file/Sparse/FRR/n=1000d=15_8778.graph
It cost 0.897612 seconds
DONE.
./graph_file/Sparse/FRR/n=1000d=6_8853.graph
It cost 0.175761 seconds
DONE.
./graph_file/Sparse/FRR/n=500d=6_34660.graph
It cost 25.763210 seconds
DONE.
./graph_file/Sparse/FRR/n=3000d=12_959.graph
It 

It cost 1.222932 seconds
DONE.
./graph_file/Sparse/FDGE/n=2000d=15_1.graph
It cost 0.735681 seconds
DONE.
./graph_file/Sparse/FDGE/n=2000d=8_1.graph
It cost 2.482922 seconds
DONE.
./graph_file/Sparse/FDGE/n=3000d=15_1.graph
It cost 0.191376 seconds
DONE.
./graph_file/Sparse/FDGE/n=1000d=8_1.graph
It cost 0.339090 seconds
DONE.
./graph_file/Sparse/FDGE/n=1000d=15_1.graph
It cost 1.296587 seconds
DONE.
./graph_file/Sparse/FDGE/n=3000d=6_1.graph
It cost 1.003030 seconds
DONE.
./graph_file/Sparse/FDGE/n=2000d=12_1.graph
It cost 0.101972 seconds
DONE.
./graph_file/Sparse/FDGE/n=500d=12_1.graph
It cost 0.611302 seconds
DONE.
./graph_file/Sparse/FDGE/n=2000d=6_1.graph
It cost 0.232508 seconds
DONE.
./graph_file/Sparse/FDGE/n=1000d=10_1.graph
It cost 0.073330 seconds
DONE.
./graph_file/Sparse/FDGE/n=500d=10_2.graph
It cost 2.060781 seconds
DONE.
./graph_file/Sparse/FDGE/n=3000d=12_1.graph
It cost 0.281662 seconds
DONE.
./graph_file/Sparse/FDGE/n=1000d=12_1.graph
It cost 0.060332 seconds
DONE.
